##### Примерное содержание первого семестра:

От исходного кода до исполняемого файла:
* что есть программа на С++
* процесс сборки 
* компилятор и некоторые его опции

Фичи языка:
* функции и операторы
* классы
* шаблоны
* исключения
* умные указатели
* стандартные контейнеры и алгоритмы
* лямбда-функции

Практики разработки:
* экосистема С++ проектов
* практики производительности
* юнит-тестирование: как и чем
* анализаторы и санитары
* паттерны проектирования

<br />

## Лекция 1. Введение

Обзор места языка на рынке
* Зачем С++? Почему не язык X?
    * Высокоуровневый язык с прямым отображением программы на железо
        * нет прослойки типа виртуальной машины или интерпретатора
        * C++ vs Python?
        * С++ vs Java? Можно побыстрее, пожалуйста?
    * Программист по умолчанию умный
* Какова ниша С++?
    * Примеры областей для использования:
        * Игровые движки
        * Физические движки
        * Браузеры
        * Компиляторы
        * Backend
        * ОС
        * Математические пакеты
        * HPC
        * ...
    * Что их объединяет?
* Альтернативы на его нише?
    * Rust

<br />

Hello world на С++ и что есть что

```c++
#include <cstdio>

int main()
{
    std::puts("Hello world");
    return 0;
}
```

<br />

От текста программы к исполняемому файлу. Зачем нужен компилятор.

In [8]:
!cat hello_world.cpp

#include <cstdio>

int main()
{
    std::puts("Hello world");
    return 0;
}



Шаг 1: компилятор преобразует файл программы на С++ в объектный файл - скомпилированный набор функций файла в виде инструкций ассемблера и констант

In [17]:
!clang++ -c -O3 hello_world.cpp -o hello_world.o

Посмотрим на содержимое `hello_world.o`. Для этого воспользуемся утилитой `objdump`.

In [18]:
!objdump -s hello_world.o


hello_world.o:     file format elf64-x86-64

Contents of section .text:
 0000 50bf0000 0000e800 00000031 c059c3    P..........1.Y. 
Contents of section .rodata.str1.1:
 0000 48656c6c 6f20776f 726c6400           Hello world.    
Contents of section .comment:
 0000 00636c61 6e672076 65727369 6f6e2036  .clang version 6
 0010 2e302e30 2d317562 756e7475 32202874  .0.0-1ubuntu2 (t
 0020 6167732f 52454c45 4153455f 3630302f  ags/RELEASE_600/
 0030 66696e61 6c2900                      final).         
Contents of section .eh_frame:
 0000 14000000 00000000 017a5200 01781001  .........zR..x..
 0010 1b0c0708 90010000 14000000 1c000000  ................
 0020 00000000 0f000000 00410e10 00000000  .........A......


Попросим `objdump` показать ассемблер секции с исполняемым кодом:

In [21]:
!objdump -d hello_world.o


hello_world.o:     file format elf64-x86-64


Disassembly of section .text:

0000000000000000 <main>:
   0:	50                   	push   %rax
   1:	bf 00 00 00 00       	mov    $0x0,%edi
   6:	e8 00 00 00 00       	callq  b <main+0xb>
   b:	31 c0                	xor    %eax,%eax
   d:	59                   	pop    %rcx
   e:	c3                   	retq   


Шаг 2: линковщик собирает несколько объектных файлов в один исполняемый

In [26]:
!clang++ -O3 hello_world.o -o hello_world.exe

Запустим:

In [23]:
!./hello_world.exe

Hello world


Подчистим

In [27]:
!rm -f *.o *.exe

Как это сделано в Java? В python?

<br />

Базовые типы данных

```c++
// https://en.cppreference.com/w/cpp/language/types

int main()
{
    // boolean
    bool b = true;
    
    // integer
    {
        // >= 16 byte
        short s = 0;
        unsigned short us = 0;
        
        // >= 16 byte
        int x = 0;
        unsigned int ux = 0;
        
        // >= 32 byte
        long l = 0;
        unsinged long ul = 0;
        
        // >= 64 byte
        long long ll = 0;
        unsigned long long ull = 0;
        
        std::int32_t i32 = 0;
        std::int64_t i64 = 0;
        std::uint32_t u32 = 0;
        std::uint64_t u64 = 0;
    }
    
    // chars
    {
        char c = 0;
        unsinged uc = 0;
        
        std::char16_t c16 = 0;
        std::char32_t c32 = 0;
    }
    
    // 1 == sizeof(char) <= sizeof(short) <= sizeof(int) <= sizeof(long) <= sizeof(long long)
    
    // floats
    {
        float f = 0.f;        // usually 32-bit IEEE-754 type
        double d = 0.;        // usually 64-bit IEEE-754 type
        long double ld = 0.;  // usually 80-bit
    }
}
```

Пределы представления чисел:

```c++
// https://en.cppreference.com/w/cpp/types/numeric_limits
    
#include <numeric_limits>
    
std::numeric_limits<std::int32_t>::max()
std::numeric_limits<std::int64_t>::min()
    
std::numeric_limits<float>::max()
std::numeric_limits<float>::min()
std::numeric_limits<float>::lowest()
```

<br />

Управляющие конструкции языка

```c++
if (boolean expression)
{
    ...
}
else
{
    ...
}

if (boolean expression)
{
    ...
}

if (boolean expression)
    operation;

    
if (int x = calculate_some_number())
{
    ... // do something with x != 0
}

if (int x = calculate_some_number(); x >= 5)
{
    ... // do something with x >= 5
}
```

```c++
// for loop
for (int i = 0; i < 100; ++i)
{
    ...
}

// range for
std::vector v = {10, 20, 30, 40, 50};
for (int i : v)
{
    ...
}

// while
while (exression)
{
    ...
}

// do-while
do
{
    ...
}
while (expression);
```

<br />

Стек и куча

![img](stack_and_heap.png "Стек и куча")


```c++
void my_function()
{
    int x = 500;  // value on stack
    int *p = &x;  // pointer to value (что такое указатель?)
    
    *p = 42;
    std::cout << x;  // ??
}
```

```c++
void my_function()
{
    int *p = (int *)malloc(sizeof(int));  // pointer to value on heap
    *p = 500;
    int x = *p;                    // value on stack

    *p = 42;
    std::cout << x;  // ??
    
    free(p);
}
```

<br />

Указатели и строки

```c++
//
// С-строки
//
// функции стандартной библиотеки для работы со строками в стиле С
// https://en.cppreference.com/w/cpp/string/byte

const char* cname = "cpushkin";  // where? len?
std::cout << cname << std::endl;

char* pcname = (char*)malloc(sizeof(char) * (strlen(cname) + 1));  // where? +1?
strcpy(pcname, cname);
std::cout << pcname << std::endl;
free(pcname);

const char* greeting = "Ai da " + cname; // ! ERROR

char s[100];
sprintf(s, "%i", 42);


//
// C++ - строки
//
// класс стандартной строки:
// https://en.cppreference.com/w/cpp/string/basic_string
//
// функции конвертации строк:
// https://en.cppreference.com/w/cpp/string/basic_string/to_string

std::string cppname = "cpppushkin";
std::cout << cppname << std::endl;
                
std::string prep = "Ai da";
std::string greeting = prep + " " + cppname; // OK
        
std::string s = std::to_string(42);

std::string* pcppname = &cppname; // what is it?
```

<br />

##### Функции

Напишем функцию, вычисляющую длину двумерного вектора:

Подробно рассказать что есть что в этом коде.

```c++
float length(float x, float y)
{
    float sqrLength = x * x + y * y;
    return std::sqrt(sqrLength);
}
```

Вызов функции:

```c++
float len = length(3.f, 4.f);
```

Если функция не должна ничего возвращать, есть специально зарезервированное слово `void`:

```c++
void log_person_arrived_to_airport(const char* name)
{
    if (!is_logging_enabled())
        return;
    
    std::clog << name << " is arrived to airport" << std::endl;
}
```

<br />

##### Передача аргументов в функцию

По значению:

```c++
void f(int n)
{
    ++n;
    std::clog << n << std::endl;
}


x = 3;
f(x);  // 4
std::cout << x << std::endl;  // 3
```

По ссылке:

```c++
void f(int& n)
{
    ++n;
    std::clog << n << std::endl;
}


x = 3;
f(x);  // 4
std::cout << x << std::endl;  // 4
```

Вариант - по константной ссылке:

```c++
void f(const int& n)
{
    ++n;  // COMPILER ERROR!
    std::clog << n << std::endl;
}


x = 3;
f(x);
std::cout << x << std::endl;
```

<br />

В каком случае лучше передвать по значению, а когда по ссылке?
* дешёвые для копирования объекты - по значению
* дорогие для копирования объекты - по ссылке

Рассмотрим разницу:

```c++
std::string get_full_name(const std::string& name, const std::string& surname)
{
    return name + " " + surname;
}

std::string get_full_name(std::string name, std::string surname)
{
    return name + " " + surname;
}
```

А если так, в чём проблема?

```c++
std::string get_full_name(std::string& name, std::string& surname)
{
    return name + " " + surname;
}
```

А что происходит здесь?

```c++
std::string get_full_name(std::string *name, std::string *surname)
{
    return *name + " " + *surname;
}
```

<br />

__Замечания по лекции__:
* Расширить рассказ про стэк и кучу. Расширить рассказ про указатели и работу с ними. Больше картинок, больше примеров. Многие из аудитории, несмотря на то что программировали на С++, не знают этих деталей.
* Добавить пример с возвращением int* на локальную переменную на стеке
* Дополнить вариант инклудов с кавычками, объяснить разницу.
* Обратить внимание какие компиляторы существуют. Некоторые не знакомы с названиями gcc/clang/msvc.
* Подробнее рассказать про цикл for